1. Import functions

In [8]:
import os
import warnings
from collections import deque
from datetime import timedelta
import random
import numpy as np
import pandas as pd
from scipy.sparse import bmat, coo_matrix, csr_matrix, diags, kron, load_npz
from scipy import linalg
import tensorflow as tf
from tensorflow.keras import Input
from tensorflow.keras.constraints import NonNeg
from tensorflow.keras.layers import Bidirectional, Dense, LSTM, TimeDistributed, Flatten, Reshape
from tensorflow.keras.metrics import *
from tensorflow.keras.models import Model
from tabulate import tabulate
from itertools import chain
from termcolor import colored
from random import randrange
from IPython.display import clear_output
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import cupy as cp
import time
import traceback
import datetime
import math
from livelossplot import PlotLossesKeras

3. Necessary functions

In [9]:
#1. folding and Unfolding a 3D tensor
def unfolding_3D(Tens, unfol_dim, other_dim_seq ):
    X = Tens.transpose(unfol_dim, other_dim_seq[0], other_dim_seq[1]).reshape(Tens.shape[unfol_dim], Tens.shape[other_dim_seq[0]] * Tens.shape[other_dim_seq[1]])
    return (X)
def folding_3D(Unfold_Tens, unfol_dim , other_dim_seq, Tens_shape):
    a = [0,1,2]
    items = deque(a)
    items.rotate(unfol_dim) 
    a_up= list(items)
    X = Unfold_Tens.reshape(Tens_shape[unfol_dim], Tens_shape[other_dim_seq[0]],Tens_shape[other_dim_seq[1]]).transpose(a_up[0], a_up[1], a_up[2])    
    return (X)
#2. ADGNTD functions
def derivative_dynamic_graph_A_W(A, B, W):
    In = csr_matrix(np.identity(A.shape[1]))
    Ip = csr_matrix(np.identity(B.shape[0]))
    Imn = csr_matrix(np.identity(A.shape[0] * A.shape[1]))
    ones = csr_matrix(np.ones((1, A.shape[0])))
    Eta_rep = csr_matrix(linalg.khatri_rao(Imn.toarray(), (B.dot(kron(In, ones))).toarray()))
    Eta = Eta_rep.T.dot(kron(In.T, W)).dot(Eta_rep)
    derivative_A = (Eta + Eta.T).dot(vect(A))
    derivative_A_mat = vec_to_Mat(derivative_A, A.shape)
    return(csr_matrix(derivative_A_mat))
def derivative_dynamic_graph_A_D(A, B, Deg):
    In = csr_matrix(np.identity(A.shape[1]))
    Ip = csr_matrix(np.identity(B.shape[0]))
    Imn = csr_matrix(np.identity(A.shape[0] * A.shape[1]))
    ones = csr_matrix(np.ones((1, A.shape[0])))
    Eta_rep = csr_matrix(linalg.khatri_rao(Imn.toarray(), (B.dot(kron(In, ones))).toarray()))
    Eta = Eta_rep.T.dot(kron(In.T, Deg)).dot(Eta_rep)
    derivative_A = (Eta + Eta.T).dot(vect(A))
    derivative_A_mat = vec_to_Mat(derivative_A, A.shape)
    return(csr_matrix(derivative_A_mat))
def derivative_dynamic_graph_B_W(A, B, W):
    In = csr_matrix(np.identity(A.shape[1]))
    Ip = csr_matrix(np.identity(B.shape[0]))
    I_dash = diags(np.ones((W.shape[0],)))
    Xi_rep = kron(linalg.khatri_rao(In.toarray(), A.toarray()), Ip)
    Xi = Xi_rep.T.dot(kron(In.T , W)).dot(Xi_rep)
    derivative_B = (Xi + Xi.T).dot(vect(B))
    derivative_B_mat = vec_to_Mat(derivative_B, B.shape)
    return(csr_matrix(derivative_B_mat))
def derivative_dynamic_graph_B_D(A, B, Deg):
    In = csr_matrix(np.identity(A.shape[1]))
    Ip = csr_matrix(np.identity(B.shape[0]))
    Xi_rep = kron(linalg.khatri_rao(In.toarray(), A.toarray()), Ip)
    Xi = Xi_rep.T.dot(kron(In.T , Deg)).dot(Xi_rep)
    derivative_B = (Xi + Xi.T).dot(vect(B))
    derivative_B_mat = vec_to_Mat(derivative_B, B.shape)
    return(csr_matrix(derivative_B_mat))
def vect(Mat):
    x = Mat.shape[0]
    y = Mat.shape[1]
    vec_A = Mat.T.reshape(x*y,1)
    return(vec_A)
def vec_to_Mat(vec, shape):
    x = shape[0]
    y = shape[1]
    Mat = vec.reshape(y,x).T
    return (Mat)
def rmse(a, b):
    rms = np.sqrt(np.power(a-b, 2).sum()/(a.shape[0]* a.shape[1]))
    return (round(rms,2))
    
def calculate_smoothed_weights(mat):
    identity_matrix = np.identity(mat.shape[0])
    W_smooth = -pd.DataFrame(identity_matrix).rolling(2, axis=1).sum().replace(np.nan, 0)
    W_smooth[0] = -pd.DataFrame(identity_matrix)[0]
    W_smooth = np.array(W_smooth + 2 * pd.DataFrame(identity_matrix))
    W_smooth_pos = (np.abs(W_smooth) + W_smooth) / 2
    W_smooth_neg = (np.abs(W_smooth) - W_smooth) / 2
    neum_term_B = csr_matrix(W_smooth_neg.T.dot(W_smooth_neg))
    denom_term_B = csr_matrix(W_smooth_pos.T.dot(W_smooth_pos))
    return  (neum_term_B, denom_term_B)

def calculate_smoothed_weights_cp(mat):
    identity_matrix = np.identity(mat.shape[0])
    W_smooth = -pd.DataFrame(identity_matrix).rolling(2, axis=1).sum().replace(np.nan, 0)
    W_smooth[0] = -pd.DataFrame(identity_matrix)[0]
    W_smooth = np.array(W_smooth + 2 * pd.DataFrame(identity_matrix))
    W_smooth_pos = (np.abs(W_smooth) + W_smooth) / 2
    W_smooth_neg = (np.abs(W_smooth) - W_smooth) / 2
    neum_term_B = cp.asarray(W_smooth_neg.T.dot(W_smooth_neg))
    denom_term_B = cp.asarray(W_smooth_pos.T.dot(W_smooth_pos))
    return  (neum_term_B, denom_term_B)

In [10]:
def ADGNTD(Tens1, W_prior, r = 10, n_iter=100, hy1=1, hy2=1, mu=0.1, factorization1=1000,
                      prin=50, A = None, B = None, C = None, W = None, delta = None ):
    T3 = csr_matrix(unfolding_3D(Tens1, unfol_dim = 2, other_dim_seq = [0,1] ))
    plot = []
    try:
        if (A == None) | (B == None)|  (C == None):
            A = csr_matrix(np.random.rand(Tens1.shape[0],r))
            B = csr_matrix(np.random.rand(Tens1.shape[1],r))
            C = csr_matrix(np.random.rand(Tens1.shape[2],r))
        if (W == None):
            W = W_prior.copy()
            delta = W*.5
        if (delta == None):
            delta = W*.5
        A_list = list(range(A.shape[0]))
        B_list = list(range(B.shape[0]))
        C_list = list(range(C.shape[0]))
        neum_term_B, denom_term_B = calculate_smoothed_weights(B)
        for indomie in range(n_iter):
            A_Batch = Tens1.shape[0] // 1
            B_Batch = Tens1.shape[1] // 1
            C_Batch = Tens1.shape[2] // 1
            A_int = random.randrange(A.shape[0])
            B_int = random.randrange(B.shape[0])
            C_int = random.randrange(C.shape[0])
            A_Sampling = random.sample(A_list, A_Batch)
            B_Sampling = random.sample(B_list, B_Batch)
            C_Sampling = random.sample(C_list, C_Batch)
            A_Sampled = csr_matrix(A[A_Sampling, :])
            B_Sampled = csr_matrix(B[B_Sampling, :])
            C_Sampled = csr_matrix(C[C_Sampling, :])
            Tens_sample = Tens1[A_Sampling, :, :][:, B_Sampling, :][:, :, C_Sampling]
            T1_sampled = csr_matrix(unfolding_3D(Tens_sample, unfol_dim=0, other_dim_seq=[1, 2]))
            T2_sampled = csr_matrix(unfolding_3D(Tens_sample, unfol_dim=1, other_dim_seq=[2, 0]))
            T3_sampled = csr_matrix(unfolding_3D(Tens_sample, unfol_dim=2, other_dim_seq=[0, 1]))
            # Loop through B_Sampling indices and create w_list_B
            for index, j in enumerate(B_Sampling):
                if index == 0:
                    w_list_B = list(range(j * A.shape[0], (j + 1) * A.shape[0]))
                if index > 0:
                    w_list_B += list(range(j * A.shape[0], (j + 1) * A.shape[0]))
            
            # Loop through B indices and create w_list_A
            for i in range(0, B.shape[0]):
                if i == 0:
                    w_list_A = list(np.array(A_Sampling) + i * A.shape[0])
                if i > 0:
                    w_list_A += list(np.array(A_Sampling) + i * A.shape[0])
            
            # Find common indices between w_list_A and w_list_B
            w_list = list(set(w_list_A) & set(w_list_B))
            # Sample submatrix W_sampled from W using w_list indices
            W_sampled = W[w_list, :][:, w_list]
            # Calculate degree matrix for sampled nodes
            Deg_sampled = sparse.diags(np.squeeze(np.asarray(W_sampled.sum(axis=0)))).tocsr()
            # Sample submatrix W_prior_sampled from W_prior using w_list indices
            W_prior_sampled = W_prior[w_list, :][:, w_list]
            # Sample submatrix delta_sampled from delta using w_list indices
            delta_sampled = delta[w_list, :][:, w_list]
            # Create matrix V using Khatri-Rao product of B_Sampled and C_Sampled
            V = csr_matrix(linalg.khatri_rao(B_Sampled.toarray(), C_Sampled.toarray()))
            # Update A_Sampled using optimization step
            A_Sampled = csr_matrix(A_Sampled.multiply(T1_sampled.dot(V) 
                                                      + hy1 * derivative_dynamic_graph_A_W(A_Sampled, B_Sampled, W_sampled)) / 
                                   (A_Sampled.dot(V.T).dot(V) + hy2 * A_Sampled + 
                                    hy1 * derivative_dynamic_graph_A_D(A_Sampled, B_Sampled, Deg_sampled)))
            # Update rows of A using A_Sampling indices
            A[A_Sampling, :] = A_Sampled
            # Recalculate matrix V using Khatri-Rao product of C_Sampled and A_Sampled
            V = csr_matrix(linalg.khatri_rao(C_Sampled.toarray(), A_Sampled.toarray()))
            # Update B_Sampled using optimization step
            B_num = B_Sampled.multiply(csr_matrix(T2_sampled.dot(V)) + 5 * (neum_term_B.dot(B)) 
                                       + hy1 * derivative_dynamic_graph_B_W(A_Sampled, B_Sampled, W_sampled))
            B_denom = (B_Sampled.dot(V.T).dot(V) + hy2 * B_Sampled + 5 * (denom_term_B.dot(B)) 
                       + hy1 * derivative_dynamic_graph_B_D(A_Sampled, B_Sampled, Deg_sampled))
            B_Sampled = csr_matrix(B_num / B_denom)
            # Update rows of B using B_Sampling indices
            B[B_Sampling, :] = B_Sampled
            # Recalculate matrix V using Khatri-Rao product of A_Sampled and B_Sampled
            V = csr_matrix(linalg.khatri_rao(A_Sampled.toarray(), B_Sampled.toarray()))
            # Update C_Sampled using optimization step
            C_Sampled = csr_matrix(C_Sampled.multiply(T3_sampled.dot(V)) / (C_Sampled.dot(V.T).dot(V) + hy2 * C_Sampled))
            # Update rows of C using C_Sampling indices
            C[C_Sampling, :] = C_Sampled
            # Estimate matrix EstimatedT4 using calculated matrices
            EstimatedT4 = C.todense().dot(linalg.khatri_rao(A.todense(), B.todense()).T)
            # Update delta_sampled with mu term
            delta_sampled = delta_sampled + mu * (delta_sampled.T - delta_sampled)
            # Update submatrix delta using delta_sampled and w_list indices
            delta[w_list, :][:, w_list] = delta_sampled
            if indomie%prin == 0:
                try:
                    factorization1 =  min(factorization, factorization1)
                except:
                    pass
                factorization = rmse(T3, EstimatedT4).round(2)
                plot.append([indomie, factorization])
                abcd = pd.DataFrame(plot, columns = ['iteration', 'RMSE'])
            if indomie%prin == 0:
                clear_output(wait=True)
                print(".............................................")
                print('iteration:                       ',indomie)
                print('rmse factorization :             ', factorization )
                try:
                    print('rmse factorization min:          ', min(factorization, factorization1))
                except:
                    pass
                print('.............................................')
    except KeyboardInterrupt:
        print("Keyboard interruption detected. Returning...")
        return(A, B, C, W, delta)
    return(A, B, C, W, delta)

def ADGNTD_stochastic(Tens1, W_prior, r = 10, iter_steps = [1000, 2000, 3000],
                      batchsize = [[20, 100, 20], [40,100,40], [60,100,60]], 
                      hy1=1, hy2=1, mu=0.1, factorization1=1000, max_iter = 6000,
                      prin=50, A = None, B = None, C = None, W = None, delta = None ):
    T3 = csr_matrix(unfolding_3D(Tens1, unfol_dim = 2, other_dim_seq = [0,1] ))
    plot = []
    try:
        if (A == None) | (B == None)|  (C == None):
            A = csr_matrix(np.random.rand(Tens1.shape[0],r))
            B = csr_matrix(np.random.rand(Tens1.shape[1],r))
            C = csr_matrix(np.random.rand(Tens1.shape[2],r))
        if (W == None):
            W = W_prior.copy()
            delta = W*.5
        if (delta == None):
            delta = W*.5
        A_list = list(range(A.shape[0]))
        B_list = list(range(B.shape[0]))
        C_list = list(range(C.shape[0]))
        neum_term_B, denom_term_B = calculate_smoothed_weights(B)
        start = 0
        for indomie in range(max_iter):
            try:
                change_iter = iter_steps[start]
            except:
                change_iter = 100000000
            if indomie >= change_iter:
                start = start+1
                #print('yes')
            if indomie <= change_iter:
                try:
                    batchsize_selected = batchsize [start]
                except:
                    pass
            #print(batchsize_selected)
            A_Batch = int(batchsize_selected[0] / 100 * Tens1.shape[0] )
            B_Batch = Tens1.shape[1]
            C_Batch = int(batchsize_selected[2] / 100 * Tens1.shape[2] )
            A_int = random.randrange(A.shape[0])
            B_int = random.randrange(B.shape[0])
            C_int = random.randrange(C.shape[0])
            A_Sampling = random.sample(A_list, A_Batch)
            B_Sampling = random.sample(B_list, B_Batch)
            C_Sampling = random.sample(C_list, C_Batch)
            A_Sampled = csr_matrix(A[A_Sampling, :])
            B_Sampled = csr_matrix(B[B_Sampling, :])
            C_Sampled = csr_matrix(C[C_Sampling, :])
            Tens_sample = Tens1[A_Sampling, :, :][:, B_Sampling, :][:, :, C_Sampling]
            T1_sampled = csr_matrix(unfolding_3D(Tens_sample, unfol_dim=0, other_dim_seq=[1, 2]))
            T2_sampled = csr_matrix(unfolding_3D(Tens_sample, unfol_dim=1, other_dim_seq=[2, 0]))
            T3_sampled = csr_matrix(unfolding_3D(Tens_sample, unfol_dim=2, other_dim_seq=[0, 1]))
            # Loop through B_Sampling indices and create w_list_B
            for index, j in enumerate(B_Sampling):
                if index == 0:
                    w_list_B = list(range(j * A.shape[0], (j + 1) * A.shape[0]))
                if index > 0:
                    w_list_B += list(range(j * A.shape[0], (j + 1) * A.shape[0]))
            
            # Loop through B indices and create w_list_A
            for i in range(0, B.shape[0]):
                if i == 0:
                    w_list_A = list(np.array(A_Sampling) + i * A.shape[0])
                if i > 0:
                    w_list_A += list(np.array(A_Sampling) + i * A.shape[0])
            
            # Find common indices between w_list_A and w_list_B
            w_list = list(set(w_list_A) & set(w_list_B))
            # Sample submatrix W_sampled from W using w_list indices
            W_sampled = W[w_list, :][:, w_list]
            # Calculate degree matrix for sampled nodes
            Deg_sampled = sparse.diags(np.squeeze(np.asarray(W_sampled.sum(axis=0)))).tocsr()
            # Sample submatrix W_prior_sampled from W_prior using w_list indices
            W_prior_sampled = W_prior[w_list, :][:, w_list]
            # Sample submatrix delta_sampled from delta using w_list indices
            delta_sampled = delta[w_list, :][:, w_list]
            # Create matrix V using Khatri-Rao product of B_Sampled and C_Sampled
            V = csr_matrix(linalg.khatri_rao(B_Sampled.toarray(), C_Sampled.toarray()))
            # Update A_Sampled using optimization step
            A_Sampled = csr_matrix(A_Sampled.multiply(T1_sampled.dot(V) 
                                                      + hy1 * derivative_dynamic_graph_A_W(A_Sampled, B_Sampled, W_sampled)) / 
                                   (A_Sampled.dot(V.T).dot(V) + hy2 * A_Sampled + 
                                    hy1 * derivative_dynamic_graph_A_D(A_Sampled, B_Sampled, Deg_sampled)))
            # Update rows of A using A_Sampling indices
            A[A_Sampling, :] = A_Sampled
            # Recalculate matrix V using Khatri-Rao product of C_Sampled and A_Sampled
            V = csr_matrix(linalg.khatri_rao(C_Sampled.toarray(), A_Sampled.toarray()))
            # Update B_Sampled using optimization step
            B_num = B_Sampled.multiply(csr_matrix(T2_sampled.dot(V)) + 5 * (neum_term_B.dot(B)) 
                                       + hy1 * derivative_dynamic_graph_B_W(A_Sampled, B_Sampled, W_sampled))
            B_denom = (B_Sampled.dot(V.T).dot(V) + hy2 * B_Sampled + 5 * (denom_term_B.dot(B)) 
                       + hy1 * derivative_dynamic_graph_B_D(A_Sampled, B_Sampled, Deg_sampled))
            B_Sampled = csr_matrix(B_num / B_denom)
            # Update rows of B using B_Sampling indices
            B[B_Sampling, :] = B_Sampled
            # Recalculate matrix V using Khatri-Rao product of A_Sampled and B_Sampled
            V = csr_matrix(linalg.khatri_rao(A_Sampled.toarray(), B_Sampled.toarray()))
            # Update C_Sampled using optimization step
            C_Sampled = csr_matrix(C_Sampled.multiply(T3_sampled.dot(V)) / (C_Sampled.dot(V.T).dot(V) + hy2 * C_Sampled))
            # Update rows of C using C_Sampling indices
            C[C_Sampling, :] = C_Sampled
            # Estimate matrix EstimatedT4 using calculated matrices
            EstimatedT4 = C.todense().dot(linalg.khatri_rao(A.todense(), B.todense()).T)
            # Update delta_sampled with mu term
            delta_sampled = delta_sampled + mu * (delta_sampled.T - delta_sampled)
            # Update submatrix delta using delta_sampled and w_list indices
            delta[w_list, :][:, w_list] = delta_sampled
            if indomie%prin == 0:
                try:
                    factorization1 =  min(factorization, factorization1)
                except:
                    pass
                factorization = rmse(T3, EstimatedT4).round(2)
                plot.append([indomie, factorization])
                abcd = pd.DataFrame(plot, columns = ['iteration', 'RMSE'])
            if indomie%prin == 0:
                clear_output(wait=True)
                print(".............................................")
                print('iteration:                       ',indomie)
                print('rmse factorization :             ', factorization )
                try:
                    print('rmse factorization min:          ', min(factorization, factorization1))
                    print('batchsize_selected:          ', batchsize_selected)
                    #batchsize_selected
                except:
                    pass
                print('.............................................')
    except KeyboardInterrupt:
        print("Keyboard interruption detected. Returning...")
        return(A, B, C, W, delta)
        
    return(A, B, C, W, delta)
    

In [11]:
#Pattern mining code
def Data_Structuring(X):
    W = np.array((X - X + 1).replace(np.nan, 0))
    RW = np.array((X - X).replace(np.nan, 1))
    Y = np.array(X.replace(np.nan, 0))
    beta = np.sqrt((Y* Y).sum().sum())
    dict_temp = {}
    parms = [W, RW, Y/beta, beta, Y]
    for i in [0,1,2,3, 4]:
        dict_temp[i] = parms[i]
    return (dict_temp)
def get_ssim(vec_1, vec_2, **kwargs):
    if type(vec_1) != type(pd.Series(dtype = int)) or type(vec_2) != type(pd.Series(dtype = int)):
        raise TypeError("Input vectors should be pandas series dataypes")
    try:
        alpha = kwargs['alpha']
    except:
        alpha = 1
        #print("Alpha value not provided, using default value (1)")
    try:
        beta = kwargs['beta']
    except:
        beta = 1
        #print("Beta value not provided, using default value (1)")
    try:
        gamma = kwargs['gamma']
    except:
        gamma = 1
        #print("Gamma value not provided, using default value (1)")
    c_1 = 0.0
    c_2 = 0.0
    c_3 = 0.0
    l = (2 * (vec_1.mean() * vec_2.mean()) + c_1) / (vec_1.mean() ** 2 + vec_2.mean()**2 + c_1)
    c = (2 * np.sqrt(vec_1.var()) * np.sqrt(vec_2.var()) + c_2) / (vec_1.var() + vec_2.var() + c_2)
    s = (((vec_1 - vec_1.mean()) * (vec_2 - vec_2.mean())).sum()/(vec_1.shape[0]-1) + c_3) / (np.sqrt(vec_1.var()) * np.sqrt(vec_2.var()) + c_3 )
    #print(l, c, s)
    #print(((2 * vec_1.mean() * vec_2.mean())*(2 *  vec_1.cov(vec_2))) / ((vec_1.mean() **2 + vec_2.mean() ** 2) * (vec_1.var() ** 2 + vec_2.var()) ))
    return (l ** alpha) * (c ** beta) * (s ** gamma)

In [12]:
#offline encoding
def r_squared(ytrue,ypred):
    if len(ytrue) == len(ypred):
        pred_mean = ypred.mean()
        var_mean = np.power((ypred-pred_mean),2)
        var_line = np.power((ytrue-ypred),2)
        r_sq = 1 - ((var_line.sum())/(var_mean.sum()))
        return r_sq
    else:
        print('Lengths Dont Match')
def initialize_weights_and_V(ind_1, day, Tens_A, data_shubham, Patterns, Patterns_B, 
                             current_day_original, time_of_day, weight_future, weight_current_pred):
    # Convert date string to datetime object
    date = datetime.datetime.strptime(ind_1.index[(day-0) * Tens_A.shape[2]: (day-0+1) * Tens_A.shape[2]][0][0], '%Y-%m-%d')
    day_of_week_current = date.strftime('%A')
    #print(data_shubham)
    # Extract test data for the current day
    test_data_current_day = data_shubham[data_shubham.index.isin(ind_1.index[(day-0) * Tens_A.shape[2]: (day-0+1) * Tens_A.shape[2]])]
    #print('a', test_data_current_day)
    current_day_comparison = test_data_current_day.reset_index(drop=True).iloc[0:time_of_day, :]
    current_day_comparison = current_day_comparison.reset_index(drop=True)
    #print('b', current_day_comparison)
    #print(Tens_A.shape)
    Selection_list = []
    # Loop through unique patterns
    for pattern in (Patterns['day_of_week']).unique():
        #print(pattern)
        Patterns_temp = Patterns.copy(deep=True)
        Patterns_temp = Patterns_temp[Patterns_temp['day_of_week'] == pattern]
        
        del Patterns_temp['day_of_week']
        del Patterns_temp['time']
        #print(Patterns_temp.shape)
        Patterns_temp = Patterns_temp.iloc[0:time_of_day, :].reset_index(drop=True)
        #print(Patterns_temp.shape, current_day_comparison.shape)
        # Calculate r_squared and average
        Selection_list.append([pattern, np.array(r_squared(Patterns_temp.unstack(0), current_day_comparison.unstack(0))).mean()])
    
    # Create a DataFrame to select the best pattern
    Selection_df = pd.DataFrame(Selection_list).sort_values(by=1, ascending=False)
    selected_pattern = Selection_df[[0]].iloc[0][0]
    train_data_shubham_temp_plot = Patterns[Patterns['day_of_week'] == selected_pattern].reset_index(drop=True)
    
    #B_half = np.array(Patterns_B[Patterns_B['cluster_id'] == selected_pattern].reset_index(drop=True).T.iloc[1:].astype('float64')).T
    #print(B_half)
    B_half = np.array(Patterns_B.mean(axis = 0).reset_index(drop=True).T.iloc[1:].astype('float64')).T
    B_half = B_half.reshape(1, B_half.shape[0])
    # Calculate current_day_original_Weights
    current_day_original_Weights = current_day_original.copy()
    current_day_original_Weights = (current_day_original_Weights + 0.00000001) / (current_day_original_Weights + 0.00000001) + weight_current_pred
    current_day_original_Weights[:, time_of_day:] = 0
    current_day_original_Weights_Opposite = current_day_original_Weights.copy()
    current_day_original_Weights_Opposite[current_day_original_Weights_Opposite == 0] = 2
    current_day_original_Weights_Opposite[current_day_original_Weights_Opposite == 1] = 0
    current_day_original_Weights_Opposite[current_day_original_Weights_Opposite == 2] = 1
    
    # Initialize V_initialize
    hist_val = cp.asarray(np.array(train_data_shubham_temp_plot[train_data_shubham_temp_plot.columns[2:]]).T)
    current_day_original[:, time_of_day + 8:] = hist_val[:, time_of_day + 8:]
    current_day_original_Weights[:, time_of_day + 8:] = weight_future
    #print(B_half.shape, C.shape)
    V_initialize_start = cp.asarray(linalg.khatri_rao(B_half, C))
    V_initialize = V_initialize_start.copy()
    current_day_original_Weights = cp.asarray(current_day_original_Weights)
    current_day_original = cp.asarray(current_day_original)
    V_initialize = cp.asarray(V_initialize)
    
    return current_day_original_Weights, V_initialize, hist_val, current_day_original


In [13]:
def Offline_encoding (Tens_A, data_shubham, A, B, C, ind,rolling_window, future_time_steps, hyper_smooth, 
                      hyp_previous_reg , weight_futuree, weight_current_pred,last_time = 88,
                      yTrain1final = None, xTrain1final = None, day_max = 20):
    print(hyp_previous_reg)
    try:
        #print('lt', last_time)
        try:
            if yTrain1final == None:
                flag = 0
            else:
                flag = 1
            day_start = 0
        except:
            flag = 1
            day_start = int(yTrain1final.shape[0] / (last_time - rolling_window))
            pass
        #print('flag...........................................................')
        # Calculate magnitude of matrix A and its inverse
        mag_A = np.diag(np.asarray(np.sqrt((A.T.dot(A)).sum(axis=1))).reshape(-1))
        mag_A_inverse = np.linalg.inv(mag_A)
        
        # Create a DataFrame 'data' using dot product and transpose
        data = pd.DataFrame((mag_A).dot(linalg.khatri_rao(B, C).T).T).round(5)
        
        # Find the maximum value in 'data'
        data_max = np.array(data).max()
        
        # Copy arrays to GPU if available
        A_dash = cp.asarray(A.copy())
        Tens_A = cp.asarray(Tens_A)
        mag_A = cp.array(mag_A)

        
        # Loop through days and time of day
        for day in range(day_start, day_max):
            a = time.time()
            print(day)
            current_day_OO = Tens_A[:, day, :].copy()
            for time_of_day in range(rolling_window, last_time):
                #print(day, time_of_day)
                #print('lt', last_time)
                # Extract current day data
                current_day_original  = current_day_OO.copy()
                #current_day_original[:, time_of_day:] = 20 + random.randint(0, 10)
                #print(current_day_original)
                # Initialize weights, V, and history value
                current_day_original_Weights, V_initialize, hist_val, current_day_original = initialize_weights_and_V(ind, 
                                                                                                                      day, Tens_A,
                                                                                                                      data_shubham, Patterns, Patterns_B, 
                                                                                                                      current_day_original, time_of_day, weight_future,weight_current_pred)
                if time_of_day == rolling_window:
                    V_initialize = V_initialize.copy()
                if time_of_day!=rolling_window:
                    V_dash.copy()
                V_initialize = cp.asarray(V_initialize)
                if (flag == 0) | (day_start!=0):
                    # Calculate numerator and denominator terms for weight smoothing
                    neum_term_C2, denom_term_C2 = calculate_smoothed_weights_cp(V_initialize)
                neum_V_new = (current_day_original_Weights.T * current_day_original.T).dot(A_dash)
                for i in range(0, 30):
                    # Update V using weighted dot products
                    
                    denom_V_new = (current_day_original_Weights.T * V_initialize.dot(A_dash.T)).dot(A_dash)
        
                    # Update V_initialize using calculated terms
                    V_initialize = V_initialize * (((neum_V_new + hyper_smooth * neum_term_C2.dot(V_initialize) + 0.000000001) /
                                                    (denom_V_new + 10 * V_initialize + 
                                                     hyper_smooth * denom_term_C2.dot(V_initialize) 
                                                     + 0.000000001)))
                #print(time_of_day)
        
                # Store previous values
                V_initialize_previous = V_initialize.copy()
                current_day_original_Weights_previous = current_day_original_Weights.copy()
        
                # Calculate V using magnitude of A and V_initialize
                V = (mag_A).dot(V_initialize.T).T/data_max
                
                # Generate training sequences
                xTrain1 = V[time_of_day - rolling_window: time_of_day, :]
                xTrain1 = xTrain1.reshape(1, *xTrain1.shape)
        
                # Concatenate or copy xTrain1_subset based on flag value
                if flag == 1:
                    xTrain1final = np.concatenate((xTrain1final, xTrain1), axis=0)
                elif flag == 0:
                    xTrain1final = xTrain1.copy()
        
                # Update weights for current day
                Weight_temp = V_initialize.copy()
                Weight_temp[time_of_day - rolling_window: time_of_day, :] = hyp_previous_reg
                Weight_temp[0: time_of_day - rolling_window, :] = 0
                Weight_temp[time_of_day:, :] = 0
        
                current_day_original  = current_day_OO.copy()
        
                # Update current day weights and values
                current_day_original_Weights = np.divide(current_day_original + 0.00000001, current_day_original + 0.00000001)
                current_day_original_Weights[:, time_of_day + future_time_steps:] = 1
                #current_day_original_Weights_Opposite = 1 - current_day_original_Weights
        
                V_initialize = V_initialize.copy()
                #current_day_original[:, time_of_day + 16:] = hist_val[:, time_of_day + 16:]
                current_day_original_Weights[:, time_of_day:] = weight_current_pred
                current_day_original_Weights[:, time_of_day:time_of_day + future_time_steps] = weight_current_pred
                t1 = (current_day_original_Weights.T * current_day_original.T).dot(A_dash) 
                for i in range(0,300):
                    # Update V using weighted dot products and smoothing terms
                    neum_V_new = t1+ Weight_temp * V_initialize_previous
                    denom_V_new = (current_day_original_Weights.T * 
                                   V_initialize.dot(A_dash.T)).dot(A_dash) +  Weight_temp * V_initialize
                    V_initialize = V_initialize * (((neum_V_new + hyper_smooth * neum_term_C2.dot(V_initialize) + 0.000000001) /
                                                    (denom_V_new + 10 * V_initialize +
                                                     hyper_smooth * denom_term_C2.dot(V_initialize) + 0.000000001)))
        
                # Update V using magnitude of A and V_initialize
                V = (mag_A).dot(V_initialize.T).T / data_max
                #print('hello', V.shape, time_of_day, time_of_day, time_of_day + future_time_steps)
                #time.sleep(5) 
                # Generate prediction sequences
                xPred1 = V[time_of_day - rolling_window: time_of_day + future_time_steps, :]
                xPred1 = xPred1.reshape(1, *xPred1.shape)
        
                # Concatenate or copy yTrain1final based on flag value
                if flag == 1:
                    #print(yTrain1final.shape, xPred1.shape)
                    yTrain1final = np.concatenate((yTrain1final, xPred1), axis=0)
                elif flag == 0:
                    yTrain1final = xPred1.copy()
                    flag = 1
                V_dash =  V_initialize_previous.copy()
                V_dash[:time_of_day, :] = V_initialize[:time_of_day,: ].copy()
                # Calculate error and print results
            #print(current_day_original_Weights[:, time_of_day:time_of_day + future_time_steps])
            err = current_day_original[:, time_of_day: time_of_day + future_time_steps] - A_dash.dot(V_initialize.T)[:, time_of_day: time_of_day + future_time_steps]
            #print(err)
            clear_output(wait=True)
            #print(current_day_original[:, time_of_day: time_of_day + future_time_steps], A_dash.dot(V_initialize.T)[:, time_of_day: time_of_day + future_time_steps])
            print(i, day, time_of_day, np.sqrt((err * err).sum() / (err.shape[0] * err.shape[1])))
    
            # Print timing and shapes
            #print(time.time() - a)
            #print(yTrain1final.shape, xTrain1final.shape, day, time_of_day, np.sqrt((err * err).sum() / (err.shape[0] * err.shape[1])))
    except KeyboardInterrupt:
        print("Keyboard interruption detected. Returning...")
        valid_entries = math.floor(yTrain1final.shape[0] / (last_time - rolling_window)) * (last_time - rolling_window)
        print('valid entrieS ',valid_entries )
        return yTrain1final[0: valid_entries, :, :], xTrain1final[0: valid_entries, :, :]
    return yTrain1final, xTrain1final

def Generate_original_sequence_arrays(Tens_A, rolling_window = 20, future_time_steps = 4, max_days = 270, last_time = 88):
    Index_final = []
    for day in range(0, max_days):
        #print(day)
        for time_of_day in range(rolling_window, 88):
            Index_final.append([day, time_of_day])
            
    flag = 11
    xTrain1true_Original = None
    yTrain1true_Original = None
    
    for day in range(0, max_days):
        #print(day)
        for time_of_day in range(20, last_time):
            day_data = Tens_A[:, day, :]  # You need to provide Tens_A
            
            #xTrain1_subset_Original = cp.asarray(day_data[:, time_of_day - rolling_window: time_of_day].T)
            #xTrain1_subset_Original = xTrain1_subset_Original.reshape(1, *xTrain1_subset_Original.shape)
            
            yTrain1_subset_Original = cp.asarray(day_data[:, time_of_day: time_of_day + future_time_steps].T)
            yTrain1_subset_Original = yTrain1_subset_Original.reshape(1, *yTrain1_subset_Original.shape)
            
            if flag == 10:
                #xTrain1true_Original = cp.concatenate((xTrain1true_Original, xTrain1_subset_Original), axis=0)
                yTrain1true_Original = cp.concatenate((yTrain1true_Original, yTrain1_subset_Original), axis=0)
            if flag == 11:
                #xTrain1true_Original = xTrain1_subset_Original.copy()
                yTrain1true_Original = yTrain1_subset_Original.copy()
                #print('Ok...............')
                flag = 10
        clear_output(wait=True)
        print(day)
        #print(day, yTrain1true_Original.shape, xTrain1true_Original.shape)
    return Index_final, yTrain1true_Original

In [14]:
def Offline_encoding_graph_stochastic(Tens_A, data_shubham, A, B, C, Deg, W, ind_1,rolling_window, future_time_steps, hyper_smooth, 
                      hyp_previous_reg , weight_future, weight_current_pred,last_time = 88,
                      yTrain1final = None, xTrain1final = None, day_max = 20, hy1 = 0.01):
    try:
        C_list = list(range(C.shape[0]))
        #print('lt', last_time)
        try:
            if yTrain1final == None:
                flag = 0
            else:
                flag = 1
            day_start = 0
        except:
            flag = 1
            day_start = int(yTrain1final.shape[0] / (last_time - rolling_window))
            pass
        #print('flag...........................................................')
        # Calculate magnitude of matrix A and its inverse
        mag_A = np.diag(np.asarray(np.sqrt((A.T.dot(A)).sum(axis=1))).reshape(-1))
        mag_A_inverse = np.linalg.inv(mag_A)
        
        # Create a DataFrame 'data' using dot product and transpose
        data = pd.DataFrame((mag_A).dot(linalg.khatri_rao(B, C).T).T).round(5)
        
        # Find the maximum value in 'data'
        data_max = np.array(data).max()
        
        # Copy arrays to GPU if available
        A_dash = cp.asarray(A.copy())
        Tens_A = cp.asarray(Tens_A)
        mag_A = cp.array(mag_A)
        complete_list = list(range(0, C.shape[0]))

        W = coo_matrix(cp.asarray(W.toarray()))
        In_n = coo_matrix(cp.identity(A_dash.shape[1]))
        I_dash_n = coo_matrix(diags(cp.ones((W.shape[0],))))
        temp_neum_term_n = coo_matrix(cp.asarray(linalg.khatri_rao(In_n.get().toarray(), A_dash.get())))
        kron1_n = kron(In_n.T , W)
        Deg = coo_matrix(cp.asarray(Deg.toarray()))
        In_d = csr_matrix(cp.identity(A_dash.shape[1]))
        khatri_rao_term_d = coo_matrix(cp.asarray(linalg.khatri_rao(In_d.get().toarray(), A_dash.get())))
        kron_term_d = kron(In_d.T , Deg)
        abcd = time.time()
        for day in range(day_start, day_max):
            a = time.time()
            #print(day)
            current_day_OO = Tens_A[:, day, :].copy()
            for time_of_day in range(rolling_window, last_time):
                a = time.time()
                #print('lt', last_time)
                # Extract current day data
                current_day_original  = current_day_OO.copy()
                #current_day_original[:, time_of_day:] = 20 + random.randint(0, 10)
                #print(current_day_original)
                # Initialize weights, V, and history value
                #print(ind_1)
                current_day_original_Weights, V_initialize, hist_val, current_day_original = initialize_weights_and_V(ind_1, 
                                                                                                                      day, Tens_A,
                                                                                                                      data_shubham, Patterns, Patterns_B, current_day_original, time_of_day, weight_future)
                
                V_initialize = cp.asarray(V_initialize)
                if (flag == 0) | (day_start!=0):
                    # Calculate numerator and denominator terms for weight smoothing
                    neum_term_C2, denom_term_C2 = calculate_smoothed_weights_cp(V_initialize)
                for i in range(0, 2):
                    # Update V using weighted dot products
                    neum_V_new = (current_day_original_Weights.T * current_day_original.T).dot(A_dash)
                    denom_V_new = (current_day_original_Weights.T * V_initialize.dot(A_dash.T)).dot(A_dash)
        
                    # Update V_initialize using calculated terms
                    V_initialize = V_initialize * (((neum_V_new + hyper_smooth * neum_term_C2.dot(V_initialize) + 0.000000001) /
                                                    (denom_V_new + 10 * V_initialize + 
                                                     hyper_smooth * denom_term_C2.dot(V_initialize) 
                                                     + 0.000000001)))
                neum_V_new = (current_day_original_Weights.T * current_day_original.T).dot(A_dash)
                for i in range(0, 30):
                    current_list = list(range(time_of_day -rolling_window, time_of_day))
                    random_numbers = random.sample(complete_list, 15)
                    combined_set = sorted(set(current_list + random_numbers))  # Combine lists and convert to set
                    range_list = [list(range(A.shape[0] * num, A.shape[0] * (num + 1))) for num in combined_set]
                    new_list = list(chain.from_iterable(range_list))
                    W_sample = coo_matrix(W.toarray()[new_list, : ][:, new_list])
                    Deg_sample = diags(cp.squeeze(cp.asarray(W_sample.sum(axis = 0)))).tocsr()
                    
                    denom_V_new = (current_day_original_Weights[:, combined_set].T * V_initialize[combined_set, :].dot(A_dash.T)).dot(A_dash)
                    I_dash_n_sample = coo_matrix(diags(cp.ones((W_sample.shape[0],))))
                    kron1_n_sample = kron(In_n.T , W_sample)
                    neum_graph = cp.asarray(hy1 *  derivative_dynamic_graph_B_W_cp(coo_matrix(A_dash),
                                                                                        coo_matrix(V_initialize[combined_set, :]),W_sample,In_n, I_dash_n_sample,temp_neum_term_n,kron1_n_sample ).toarray())
                    kron_term_d_sample = kron(In_d.T , Deg_sample)
                    denom_graph = cp.asarray(hy1 * derivative_dynamic_graph_B_D_cp(coo_matrix(A_dash),
                                                                                   coo_matrix(V_initialize[combined_set, :]), Deg_sample,In_d,khatri_rao_term_d, kron_term_d_sample ).toarray())
                    V_initialize[combined_set, :] = V_initialize[combined_set, :] * (((neum_V_new[combined_set, :] + hyper_smooth * neum_term_C2[combined_set, :][:, combined_set].dot(V_initialize[combined_set, :]) + 0.000000001 + neum_graph) /
                                                        (denom_V_new + 100 * V_initialize[combined_set, :] + 
                                                         hyper_smooth * denom_term_C2[combined_set, :][:, combined_set].dot(V_initialize[combined_set, :]) 
                                                         + 0.000000001 + denom_graph)))
                #err = current_day_original[:, time_of_day-4: time_of_day ] - A_dash.dot(V_initialize.T)[:, time_of_day-4: time_of_day]
                #print(day, time_of_day, np.sqrt((err * err).sum() / (err.shape[0] * err.shape[1])), time.time() - a)
                V_initialize_previous = V_initialize.copy()
                current_day_original_Weights_previous = current_day_original_Weights.copy()
        
                # Calculate V using magnitude of A and V_initialize
                V = (mag_A).dot(V_initialize.T).T/data_max
                
                # Generate training sequences
                xTrain1 = V[time_of_day - rolling_window: time_of_day, :]
                xTrain1 = xTrain1.reshape(1, *xTrain1.shape)
        
                # Concatenate or copy xTrain1_subset based on flag value
                if flag == 1:
                    xTrain1final = np.concatenate((xTrain1final, xTrain1), axis=0)
                elif flag == 0:
                    xTrain1final = xTrain1.copy()
        
                # Update weights for current day
                Weight_temp = V_initialize.copy()
                Weight_temp[0: time_of_day - rolling_window] = 0
                Weight_temp[time_of_day:] = 0
        
                current_day_original  = current_day_OO.copy()
        
                # Update current day weights and values
                current_day_original_Weights = np.divide(current_day_original + 0.00000001, current_day_original + 0.00000001)
                current_day_original_Weights[:, time_of_day + future_time_steps:] = 1
                #current_day_original_Weights_Opposite = 1 - current_day_original_Weights
        
                V_initialize = V_initialize_previous.copy()
                current_day_original[:, time_of_day + 16:] = hist_val[:, time_of_day + 16:]
                current_day_original_Weights[:, time_of_day + 16:] = weight_future
                current_day_original_Weights[:, time_of_day:time_of_day + future_time_steps] = weight_current_pred
                for i in range(0,2):
                    # Update V using weighted dot products and smoothing terms
                    neum_V_new = (current_day_original_Weights.T * current_day_original.T).dot(A_dash) + hyp_previous_reg * Weight_temp * V_initialize_previous
                    denom_V_new = (current_day_original_Weights.T * V_initialize.dot(A_dash.T)).dot(A_dash) + hyp_previous_reg * Weight_temp * V_initialize
                    V_initialize = V_initialize * (((neum_V_new + hyper_smooth * neum_term_C2.dot(V_initialize) + 0.000000001) /
                                                    (denom_V_new + 10 * V_initialize + hyper_smooth * denom_term_C2.dot(V_initialize) + 0.000000001)))
        
                neum_V_new = (current_day_original_Weights.T * current_day_original.T).dot(A_dash) + hyp_previous_reg * Weight_temp * V_initialize_previous
                
                for i in range(0, 30):
                    current_list = list(range(time_of_day -rolling_window, time_of_day + future_time_steps))
                    random_numbers = random.sample(complete_list, 15)
                    combined_set = sorted(set(current_list + random_numbers))  # Combine lists and convert to set
                    range_list = [list(range(A.shape[0] * num, A.shape[0] * (num + 1))) for num in combined_set]
                    new_list = list(chain.from_iterable(range_list))
                    W_sample = coo_matrix(W.toarray()[new_list, : ][:, new_list])
                    Deg_sample = diags(cp.squeeze(cp.asarray(W_sample.sum(axis = 0)))).tocsr()
                    
                    denom_V_new = (current_day_original_Weights[:, combined_set].T * V_initialize[combined_set, :].dot(A_dash.T)).dot(A_dash) +  hyp_previous_reg * Weight_temp[combined_set, :] * V_initialize[combined_set, :]
                    #denom_V_new = (current_day_original_Weights.T * V_initialize.dot(A_dash.T)).dot(A_dash) + hyp_previous_reg * Weight_temp * V_initialize
                    I_dash_n_sample = coo_matrix(diags(cp.ones((W_sample.shape[0],))))
                    kron1_n_sample = kron(In_n.T , W_sample)
                    neum_graph = cp.asarray(hy1 *  derivative_dynamic_graph_B_W_cp(coo_matrix(A_dash),
                                                                                        coo_matrix(V_initialize[combined_set, :]),W_sample,In_n, I_dash_n_sample,temp_neum_term_n,kron1_n_sample ).toarray())
                    kron_term_d_sample = kron(In_d.T , Deg_sample)
                    denom_graph = cp.asarray(hy1 * derivative_dynamic_graph_B_D_cp(coo_matrix(A_dash),
                                                                                   coo_matrix(V_initialize[combined_set, :]), Deg_sample,In_d,khatri_rao_term_d, kron_term_d_sample ).toarray())
                    V_initialize[combined_set, :] = V_initialize[combined_set, :] * (((neum_V_new[combined_set, :] + hyper_smooth * neum_term_C2[combined_set, :][:, combined_set].dot(V_initialize[combined_set, :]) + 0.000000001 + neum_graph) /
                                                        (denom_V_new + 100 * V_initialize[combined_set, :] + 
                                                         hyper_smooth * denom_term_C2[combined_set, :][:, combined_set].dot(V_initialize[combined_set, :]) 
                                                         + 0.000000001 + denom_graph)))
        
                #err = current_day_original[:, time_of_day: time_of_day +4 ] - A_dash.dot(V_initialize.T)[:, time_of_day: time_of_day +4]
                #print(day, time_of_day, np.sqrt((err * err).sum() / (err.shape[0] * err.shape[1])), time.time() - a)
                V = (mag_A).dot(V_initialize.T).T / data_max
                #print('hello', V.shape, time_of_day, time_of_day, time_of_day + future_time_steps)
                #time.sleep(5) 
                # Generate prediction sequences
                xPred1 = V[time_of_day - rolling_window: time_of_day + future_time_steps, :]
                xPred1 = xPred1.reshape(1, *xPred1.shape)
        
                # Concatenate or copy yTrain1final based on flag value
                if flag == 1:
                    #print(yTrain1final.shape, xPred1.shape)
                    yTrain1final = np.concatenate((yTrain1final, xPred1), axis=0)
                elif flag == 0:
                    yTrain1final = xPred1.copy()
                    flag = 1
        
                # Calculate error and print results
                #print(current_day_original_Weights[:, time_of_day:time_of_day + future_time_steps])
                if time_of_day%1 == 0:
                    err = current_day_original[:, time_of_day: time_of_day + future_time_steps] - A_dash.dot(V_initialize.T)[:, time_of_day: time_of_day + future_time_steps]
                    #print(err)
                    clear_output(wait=True)
                    #print(current_day_original[:, time_of_day: time_of_day + future_time_steps], A_dash.dot(V_initialize.T)[:, time_of_day: time_of_day + future_time_steps])
                    print(day, time_of_day, np.sqrt((err * err).sum() / (err.shape[0] * err.shape[1])), time.time() - abcd)
    
        # Print timing and shapes
        #print(time.time() - a)
        #print(yTrain1final.shape, xTrain1final.shape, day, time_of_day, np.sqrt((err * err).sum() / (err.shape[0] * err.shape[1])))
    except KeyboardInterrupt:
        print("Keyboard interruption detected. Returning...")
        valid_entries = math.floor(yTrain1final.shape[0] / (last_time - rolling_window)) * (last_time - rolling_window)
        print('valid entrieS ',valid_entries )
        return yTrain1final[0: valid_entries, :, :], xTrain1final[0: valid_entries, :, :]
    return yTrain1final, xTrain1final

hy1 = 0.001
flag = 0
hy_prev = 100
from cupyx.scipy.sparse import csr_matrix, diags
for day in range(0, 1):
    a = time.time()
    current_day_OO = Tens[:, day, :].copy()
    e = 0.00000001
    A_cp = cp.asarray(A.copy())
    for time_of_day in range(rolling_window, last_time):
        for iteration in range(0, 25):
            if iteration == 0:
                rolling_window_array = cp.asarray(current_day_OO[:, time_of_day-rolling_window:time_of_day])
                V_int = cp.random.rand(rolling_window_array.shape[1],r)
                W_temp = W[:, (time_of_day - rolling_window)* Tens.shape[0]: time_of_day* Tens.shape[0]]
                W_temp = W_temp[(time_of_day - rolling_window)* Tens.shape[0]: time_of_day* Tens.shape[0], :]
                Deg_temp = Deg[:, (time_of_day - rolling_window)* Tens.shape[0]: time_of_day* Tens.shape[0]]
                Deg_temp = Deg_temp[(time_of_day - rolling_window)* Tens.shape[0]: time_of_day* Tens.shape[0], :]
                kron_product = generate_graph_functions_inside_loop_once(In, W_temp)
            if iteration >=0:
                Ip, Xi_rep = generate_graph_functions_inside_loop(V_int, In, A_cp)
                neum_term1 = rolling_window_array.T.dot(A_cp) + hy1 * derivative_dynamic_graph_B_W_cp(A_cp, V_int, W_temp, In, Ip, I_dash, kron_product, Xi_rep)
                denom_term1 = (V_int.dot(A_cp.T)).dot(A_cp) + hy1 * derivative_dynamic_graph_B_D_cp(A_cp, V_int, Deg_temp, In, Ip, Xi_rep)
                V_int = V_int * (neum_term1 + e )/ (denom_term1 + e)
        print(time_of_day, time.time() - a, calculate_rmse(rolling_window_array, A_cp.dot(V_int.T)))
        V_initialize_previous = V_int.copy()
        V = (cp.asarray(mag_A)).dot(V_int.T).T/data_max
        xTrain1 = V[:, :]
        xTrain1 = xTrain1.reshape(1, *xTrain1.shape)
        if flag == 1:
            xTrain1final = np.concatenate((xTrain1final, xTrain1), axis=0)
        elif flag == 0:
            xTrain1final = xTrain1.copy()
        
        for iteration in range(0, 40):
            if iteration == 0:
                rolling_window_array = cp.asarray(current_day_OO[:, time_of_day-rolling_window:time_of_day + future_time_steps])
                V_int = cp.random.rand(rolling_window_array.shape[1],r)
                V_int[0:rolling_window, ] = V_initialize_previous
                V_int_prev = V_int.copy()
                W_temp = W[:, (time_of_day - rolling_window)* Tens.shape[0]: (time_of_day +future_time_steps) * Tens.shape[0]]
                W_temp = W_temp[(time_of_day - rolling_window)* Tens.shape[0]: (time_of_day +future_time_steps) * Tens.shape[0], :]
                Deg_temp = Deg[:, (time_of_day - rolling_window)* Tens.shape[0]: (time_of_day +future_time_steps) * Tens.shape[0]]
                Deg_temp = Deg_temp[(time_of_day - rolling_window)* Tens.shape[0]: (time_of_day +future_time_steps) * Tens.shape[0], :]
                kron_product = generate_graph_functions_inside_loop_once(In, W_temp)
                Weight_prev = (V_int + 0.0001)/(V_int + 0.0001)
                Weight_prev[time_of_day:] = 0
                
            if iteration >=0:
                Ip, Xi_rep = generate_graph_functions_inside_loop(V_int, In, A_cp)
                neum_term1 = (rolling_window_array.T.dot(A_cp) 
                              + hy1 * derivative_dynamic_graph_B_W_cp(A_cp, V_int, W_temp, In, Ip, I_dash2, kron_product, Xi_rep)
                             + hy_prev * Weight_prev * V_int_prev)
                denom_term1 = ((V_int.dot(A_cp.T)).dot(A_cp) 
                               + hy1 * derivative_dynamic_graph_B_D_cp(A_cp, V_int, Deg_temp, In, Ip, Xi_rep)
                              + hy_prev * Weight_prev * V_int)
                V_int = V_int * (neum_term1 + e )/ (denom_term1 + e)
        V = (cp.asarray(mag_A)).dot(V_int.T).T / data_max
        xPred1 = V[:, :]
        xPred1 = xPred1.reshape(1, *xPred1.shape)
        if flag == 1:
            yTrain1final = np.concatenate((yTrain1final, xPred1), axis=0)
        elif flag == 0:
            yTrain1final = xPred1.copy()
            flag = 1

In [15]:

def Khatri_Rao_Product(A1, B1):
    result_list = [i * A1.shape[1] + i for i in range(B1.shape[1])]
    temp = csr_matrix(kron(A1, B1))
    temp = temp[:,result_list ]
    return(temp)
def vec_to_Mat(vec, shape):
    x = shape[0]
    y = shape[1]
    Mat = vec.reshape(y,x).T
    return (Mat)
def derivative_dynamic_graph_B_W_cp(A_cp, V_int, W_temp, In, Ip, I_dash, kron_product, Xi_rep):
    Xi = Xi_rep.T.dot(kron_product).dot(Xi_rep)
    derivative_B = (Xi + Xi.T).dot(vect(V_int))
    derivative_B_mat = vec_to_Mat(derivative_B, V_int.shape)
    return(csr_matrix(derivative_B_mat))
def derivative_dynamic_graph_B_D_cp(A_cp, V_int, Deg_temp, In, Ip, Xi_rep):
    Xi = Xi_rep.T.dot(kron(In.T , Deg_temp)).dot(Xi_rep)
    derivative_B = (Xi + Xi.T).dot(vect(V_int))
    derivative_B_mat = vec_to_Mat(derivative_B, V_int.shape)
    return(csr_matrix(derivative_B_mat))

def generate_graph_functions_outside_loop(A_cp, length, future_time_steps):
    scipy_csr_identity = identity(A_cp.shape[0] * length, format='csr')
    I_dash = cp.sparse.csr_matrix(scipy_csr_identity)
    In = csr_matrix(cp.identity(A_cp.shape[1]))
    scipy_csr_identity2 = identity(A.shape[0] * (length + future_time_steps), format='csr')
    I_dash2 = cp.sparse.csr_matrix(scipy_csr_identity2)
    return (In, I_dash, I_dash2)

def generate_graph_functions_inside_loop_once(In, W_temp):
    kron_product = kron(In.T , W_temp)
    return(kron_product)

def generate_graph_functions_inside_loop(V_int, In, A_cp):
    Ip = csr_matrix(cp.identity(V_int.shape[0]))
    Xi_rep = kron(Khatri_Rao_Product(In, A_cp), Ip)
    return (Ip, Xi_rep)
def calculate_rmse(y_true, y_pred):
    squared_diff = (y_true - y_pred) ** 2
    mean_squared_diff = cp.mean(squared_diff)
    rmse = cp.sqrt(mean_squared_diff)
    return rmse.item()

In [16]:
from scipy.sparse import identity
from cupyx.scipy.sparse import kron

In [18]:
import os
from scipy import sparse
# Set up data directory
name = 'Logan'
data_directory = r'\\hpc-fs.qut.edu.au\n10680535\Paper_5\Datasets'

# Load data
Tens = np.load(os.path.join(data_directory, f'Train_{name}_3D_data.npy'))
Test_Tens = np.load(os.path.join(data_directory, f'Test_{name}_3D_data.npy'))
test_dataset = pd.read_csv(os.path.join(data_directory, f'test_{name}_data.csv'), index_col=[0, 1], header=[0])
Val_Tens = np.load(os.path.join(data_directory, f'Val_{name}_3D_data.npy'))
val_dataset = pd.read_csv(os.path.join(data_directory, f'Val_{name}_data.csv'), index_col=[0, 1], header=[0])
ind = pd.read_csv(os.path.join(data_directory, 'date_time_train.csv'), index_col=[0, 1])
stl_clm = pd.read_csv(os.path.join(data_directory, 'section_names.csv'), index_col=0)
train_data_shubham = pd.read_csv(os.path.join(data_directory, f'Train_{name}_data.csv'), index_col=[0, 1], header=[0])
W = sparse.load_npz(os.path.join(data_directory, f'Dynamic_Adjacency_{name}.npz'))
MWY_Sections = pd.read_csv(os.path.join(data_directory, 'MWY_Sections.csv'))

# Extract section IDs
section_ids = train_data_shubham.columns.to_list()
serial_numbers = [test_dataset.columns.get_loc(section_id) for section_id in section_ids]

# Manipulate data based on section IDs
Tens = Tens[serial_numbers, :, :]
Test_Tens = Test_Tens[serial_numbers, :, :]
Val_Tens = Val_Tens[serial_numbers, :, :]
stl_clm = stl_clm.iloc[serial_numbers]
train_data_shubham = train_data_shubham.T.iloc[serial_numbers].T
test_dataset = test_dataset.T.iloc[serial_numbers].T
val_dataset = val_dataset.T.iloc[serial_numbers].T

# Additional data manipulations
stl_clm.index = stl_clm.index.astype('str')
test_dataset = test_dataset[stl_clm.index].T
test_data_prior = test_dataset.copy(deep=True)
val_dataset = val_dataset[stl_clm.index].T
val_data_prior = val_dataset.copy(deep=True)
test_dataset_col = test_dataset.columns
val_dataset_col = val_dataset.columns
train_data_shubham = train_data_shubham[test_data_prior.index.astype(str)]
train_data_shubham_prior = train_data_shubham.T

# Update serial numbers for W
serial_numbers_W = serial_numbers.copy()
for i in range(1, 96):
    for j in serial_numbers:
        serial_numbers_W.append(i * 476 + j)

# Update adjacency matrix W
W = coo_matrix(W.toarray()[serial_numbers_W].T[serial_numbers_W].T)
W = coo_matrix(W)
W_dense = W.toarray()
W = (W + W.T) / 2
W.setdiag(0, k=0)
Deg = sparse.diags(np.squeeze(np.asarray(W.sum(axis=0)))).tocsr()

# File path for complete data
file_path = os.path.join(data_directory, 'Complete_Logan_3D_data.npy')
Complete_Tens = np.load(file_path)
Complete_Tens = Complete_Tens[serial_numbers, :, :]

# Additional data transformation
Tens1 = Tens.transpose(0, 2, 1)

In [ ]:
#link = r'\\hpc-fs.qut.edu.au\n10680535\Paper_5\5'
A = np.load('A.npy')
B = np.load('B.npy')
C = np.load('C.npy')
A.shape
flag = 0
rolling_window = 12
future_time_steps = 4
last_time = 88
hyper_smooth = 0
weight_future = 0.001
hyp_x =hyp_previous_reg= 2000
weight_current_pred = 20000
mag_A = np.diag(np.asarray(np.sqrt((A.T.dot(A)).sum(axis=1))).reshape(-1))
mag_A_inverse = np.linalg.inv(mag_A)
data = pd.DataFrame((mag_A).dot(linalg.khatri_rao(B, C).T).T, index=ind.index).round(5)
data_max = np.array(data).max()
rank = r = A.shape[1]
rank

In [ ]:
A.shape, B.shape, C.shape, Tens.shape, Tens.shape

In [ ]:
hy1 = 0.1
flag = 0
hy_prev = 100
from cupyx.scipy.sparse import csr_matrix, diags
for day in range(0, 20):
    a = time.time()
    current_day_OO = Tens[:, day, :].copy()
    e = 0.00000001
    
    for time_of_day in range(rolling_window, last_time):
        for iteration in range(0, 25):
            A_cp = cp.asarray(A.copy())
            if iteration == 0:
                rolling_window_array_complete = cp.asarray(current_day_OO[:, time_of_day-rolling_window:time_of_day])
                V_int = cp.random.rand(rolling_window_array_complete.shape[1],A.shape[1])
                W_temp_complete = W[:, (time_of_day - rolling_window)* Tens.shape[0]: time_of_day* Tens.shape[0]]
                W_temp_complete = W_temp_complete[(time_of_day - rolling_window)* Tens.shape[0]: time_of_day* Tens.shape[0], :]
                Deg_temp_complete = Deg[:, (time_of_day - rolling_window)* Tens.shape[0]: time_of_day* Tens.shape[0]]
                Deg_temp_complete = Deg_temp_complete[(time_of_day - rolling_window)* Tens.shape[0]: time_of_day* Tens.shape[0], :]
            random_numbers = sorted([random.randint(0, Tens.shape[0]-1) for _ in range(60)])
            list_final = []
            for ii in range (0, rolling_window):
                for jj in random_numbers:
                    list_final.append(Tens.shape[0] * ii + jj)
            A_cp = A_cp[random_numbers, :]
            In, I_dash, I_dash2 = generate_graph_functions_outside_loop(A_cp, length = rolling_window, future_time_steps= future_time_steps)
            rolling_window_array =  rolling_window_array_complete[random_numbers, :]
            W_temp = W_temp_complete[list_final, :]
            W_temp = W_temp[:, list_final]
            Deg_temp = csr_matrix(cp.diag(cp.sum(W_temp.toarray(), axis=1).ravel()))
            #In, I_dash, I_dash2 = generate_graph_functions_outside_loop(A_cp, length = rolling_window, future_time_steps= future_time_steps)
            kron_product = generate_graph_functions_inside_loop_once(In, W_temp)
            if iteration >=0:
                Ip, Xi_rep = generate_graph_functions_inside_loop(V_int, In, A_cp)
                neum_term1 = rolling_window_array.T.dot(A_cp) + hy1 * derivative_dynamic_graph_B_W_cp(A_cp, V_int, W_temp, In, Ip, I_dash, kron_product, Xi_rep)
                denom_term1 = (V_int.dot(A_cp.T)).dot(A_cp) + hy1 * derivative_dynamic_graph_B_D_cp(A_cp, V_int, Deg_temp, In, Ip, Xi_rep)
                V_int = V_int * (neum_term1 + e )/ (denom_term1 + e)
        print(time_of_day, time.time() - a, calculate_rmse(rolling_window_array_complete, cp.asarray(A.copy()).dot(V_int.T)))
        V_initialize_previous = V_int.copy()
        V = (cp.asarray(mag_A)).dot(V_int.T).T/data_max
        xTrain1 = V[:, :]
        xTrain1 = xTrain1.reshape(1, *xTrain1.shape)
        if flag == 1:
            xTrain1final = cp.concatenate((xTrain1final, xTrain1), axis=0)
        elif flag == 0:
            xTrain1final = xTrain1.copy()

In [ ]:
import multiprocessing  
import random
from multiprocessing import Pool
import defs

In [ ]:
hy1 = 0.1
flag = 0
hy_prev = 10000
from cupyx.scipy.sparse import csr_matrix, diags
for day in range(0, 20):
    a = time.time()
    current_day_OO = Tens[:, day, :].copy()
    e = 0.00000001
    W_temp_complete = W[:, (time_of_day - rolling_window)* Tens.shape[0]: time_of_day* Tens.shape[0]]
    W_temp_complete = W_temp_complete[(time_of_day - rolling_window)* Tens.shape[0]: time_of_day* Tens.shape[0], :]
    W_temp_complete2 = W[:, (time_of_day - rolling_window)* Tens.shape[0]: (time_of_day +future_time_steps) * Tens.shape[0]]
    W_temp_complete2 = W_temp[(time_of_day - rolling_window)* Tens.shape[0]: (time_of_day +future_time_steps) * Tens.shape[0], :]
    #W_temp_complete2 = W[:, (time_of_day - rolling_window)* Tens.shape[0]: (time_of_day +future_time_steps) * Tens.shape[0]]
    #W_temp_complete2 = W_temp_complete[(time_of_day - rolling_window)* Tens.shape[0]: (time_of_day +future_time_steps) * Tens.shape[0], :]
    rolling_window_array_complete = cp.asarray(current_day_OO[:, time_of_day-rolling_window:time_of_day])
    V_int = cp.random.rand(rolling_window_array_complete.shape[1],A.shape[1])
    break

In [ ]:
rank

In [ ]:
W_temp_complete2 = W[:, (time_of_day - rolling_window)* Tens.shape[0]: (time_of_day +future_time_steps) * Tens.shape[0]]
W_temp_complete2 = W_temp_complete2[(time_of_day - rolling_window)* Tens.shape[0]: (time_of_day +future_time_steps) * Tens.shape[0], :]
W_temp_complete2.shape
flag = 0
flago = 0

In [ ]:
tods = list(range(20, last_time))
hy1 = 0.0001
hy_prev = 5000
flago = 0
hy_smooth = 0
from cupyx.scipy.sparse import csr_matrix, diags
for day in range(0, 270):
    abcd = time.time()
    current_day_OO = Tens[:, day, :].copy()
    e = 0.00000001
    complete_list = []
    for indexo , time_of_day in enumerate(tods):
        W_temp_complete = W[:, (time_of_day - rolling_window)* Tens.shape[0]: time_of_day* Tens.shape[0]]
        W_temp_complete = W_temp_complete[(time_of_day - rolling_window)* Tens.shape[0]: time_of_day* Tens.shape[0], :]
        W_temp_complete2 = W[:, (time_of_day - rolling_window)* Tens.shape[0]: (time_of_day +future_time_steps) * Tens.shape[0]]
        W_temp_complete2 = W_temp_complete2[(time_of_day - rolling_window)* Tens.shape[0]: (time_of_day +future_time_steps) * Tens.shape[0], :]
        temp = []
        temp.append(time_of_day)
        temp.append(A)
        temp.append(current_day_OO)
        temp.append(W_temp_complete)
        temp.append(W_temp_complete2)
        temp.append(hy1)
        temp.append(e)
        temp.append(mag_A)
        temp.append(data_max)
        temp.append(rolling_window)
        temp.append(future_time_steps)
        temp.append(hy_prev)
        temp.append(indexo)
        temp.append(hy_smooth)
        complete_list.append(temp)
    import defs_np_new74 as defs_np
    results_x = []
    results_y = []
    results_z = []
    def collect_results(result):
        results_x.append(result[0])
        results_y.append(result[1])
        results_z.append(result[2])
    if __name__ == '__main__':
        start_time = time.time()
        pool = multiprocessing.Pool(processes = multiprocessing.cpu_count())
        print(day)
        for i_index, i in enumerate(complete_list):
            try:
                result = pool.apply_async(defs_np.process_time_of_day, args = [i[0], i[1], i[2],
                               i[3], i[4], i[5], 
                               i[6], i[7], i[8], i[9],i[10], i[11], i[12],i[13], ], callback = collect_results)
            except Exception as e:
                print(e)
                print(f"Error in worker process: {e}")
        pool.close()
        print('OK')
        pool.join()
        print('OK2')
    position_list = [results_z.index(i) for i in range(len(results_z))]
    #print(position_list)

    if flago == 1:
        x = np.concatenate(results_x)[position_list, :, :]
        y = np.concatenate(results_y)[position_list, :, :]
        xTrain1final = np.concatenate((xTrain1final,x), axis = 0)
        yTrain1final = np.concatenate((yTrain1final,y), axis = 0)
    if flago == 0:
        xTrain1final =np.concatenate(results_x)[position_list, :, :]
        yTrain1final = np.concatenate(results_y)[position_list, :, :]
        flago = 1
    print(day, xTrain1final.shape, yTrain1final.shape, time.time()-abcd)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# Example loop to generate plots
for i in range(0, 10):
    fig, ax = plt.subplots()
    y_train_df = pd.DataFrame(xTrain1final[30])
    x_train_df = pd.DataFrame(yTrain1final[30])
    pd.DataFrame({'V-output sequence': y_train_df[i]}).plot(ax=ax, color='red', label='V-output sequence')
    pd.DataFrame({'V-output sequence': x_train_df[i]}).plot(ax=ax, label='V-input sequence')
    plt.xlabel('time-period')  # Set x-axis label
    plt.ylabel('coefficient')  # Set y-axis label
    plt.title('Sequences')  # Set plot title
    plt.legend()  # Display legend
    plt.show()


In [ ]:
rec = A.dot(Khatri_Rao_Product(csr_matrix(cp.asarray(B)), csr_matrix(cp.asarray(C))).get().toarray().T)

In [ ]:
rec = pd.DataFrame(rec, index = train_data_shubham.columns, columns = train_data_shubham.index)

In [ ]:
rec

In [ ]:
# Provide the values for rolling_window and future_time_steps
rolling_window = 12
future_time_steps = 4
Index_final, yTrain1true_Original1 = Generate_original_sequence_arrays(Tens, rolling_window = 20, 
                                                                       future_time_steps = 4,
                                                                       max_days = 4, 
                                                                       last_time = 88)
yTrain1true_Original1.shape

In [ ]:
yTrain1final.shape

In [ ]:
pd.DataFrame(yTrain1true_Original1[:, 0, :].get())[90][0:68 * 1].plot()
pd.DataFrame(A.dot(mag_A_inverse.dot(yTrain1final[:, 12, :].T)) * data_max).T[90][0:68 * 1].plot(color = 'red')

In [ ]:
i

In [ ]:
flago1 = 0
rolling_window = 12
future_time_steps = 4

In [ ]:
tods = list(range(20, last_time))
from cupyx.scipy.sparse import csr_matrix, diags
for day in range(0, 30):
    abcd = time.time()
    current_day_OO = Test_Tens[:, day, :].copy()
    e = 0.00000001
    complete_list = []
    for indexo , time_of_day in enumerate(tods):
        W_temp_complete = W[:, (time_of_day - rolling_window)* Test_Tens.shape[0]: time_of_day* Test_Tens.shape[0]]
        W_temp_complete = W_temp_complete[(time_of_day - rolling_window)* Test_Tens.shape[0]: time_of_day* Test_Tens.shape[0], :]
        W_temp_complete2 = W[:, (time_of_day - rolling_window)* Test_Tens.shape[0]: (time_of_day +future_time_steps) * Test_Tens.shape[0]]
        W_temp_complete2 = W_temp_complete2[(time_of_day - rolling_window)* Test_Tens.shape[0]: (time_of_day +future_time_steps) * Test_Tens.shape[0], :]
        temp = []
        temp.append(time_of_day)
        temp.append(A)
        temp.append(current_day_OO)
        temp.append(W_temp_complete)
        temp.append(W_temp_complete2)
        temp.append(hy1)
        temp.append(e)
        temp.append(mag_A)
        temp.append(data_max)
        temp.append(rolling_window)
        temp.append(future_time_steps)
        temp.append(hy_prev)
        temp.append(indexo)
        temp.append(hy_smooth)
        complete_list.append(temp)
    results_x = []
    results_y = []
    results_z = []
    def collect_results(result):
        results_x.append(result[0])
        results_y.append(result[1])
        results_z.append(result[2])
    if __name__ == '__main__':
        start_time = time.time()
        pool = multiprocessing.Pool(processes = multiprocessing.cpu_count() - 4)
        print(day)
        for i_index, i in enumerate(complete_list):
            try:
                result = pool.apply_async(defs_np.process_time_of_day, args = [i[0], i[1], i[2],
                               i[3], i[4], i[5], 
                               i[6], i[7], i[8], i[9],i[10], i[11], i[12],i[13], ], callback = collect_results)
            except:
                print(e)
                print(f"Error in worker process: {e}")
        pool.close()
        pool.join()
    position_list = [results_z.index(i) for i in range(len(results_z))]
    #print(position_list)
    if flago1 == 1:
        x = np.concatenate(results_x)[position_list, :, :]
        y = np.concatenate(results_y)[position_list, :, :]
        xVal1final = np.concatenate((xVal1final,x), axis = 0)
        yVal1final = np.concatenate((yVal1final,y), axis = 0)
    if flago1 == 0:
        xVal1final = np.concatenate(results_x)[position_list, :, :]
        yVal1final = np.concatenate(results_y)[position_list, :, :]
        flago1 = 1
    print(day, xVal1final.shape, yVal1final.shape, time.time()-abcd)

In [ ]:
yVal1final.shape

In [ ]:
np.save('yTrain1final_g2.npy', yTrain1final)
np.save('xTrain1final_g2.npy', xTrain1final)
np.save('yVal1final_g1.npy', yVal1final)
np.save('xVal1final_g1.npy', xVal1final)

In [ ]:
defs_np.process_time_of_day(i[0], i[1], i[2],
                           i[3], i[4], i[5], 
                           i[6], i[7], i[8], i[9],i[10], i[11],)

In [ ]:
i

In [ ]:
results_x = []
results_y = []
def collect_results(result):
    results_x.append(result[0])
    results_y.append(result[1])
if __name__ == '__main__':
    start_time = time.time()
    pool = multiprocessing.Pool(processes = multiprocessing.cpu_count() -5)
    for i in complete_list:
        try:
            result = pool.apply_async(defs_np020.process_time_of_day, args = [i[0], i[1], i[2],
                           i[3], i[4], i[5], 
                           i[6], i[7], i[8], i[9],i[10], i[11],], callback = collect_results)
            print('yes')
        except:
            print(f"Error in worker process: {e}")
    pool.close()
    pool.join()

In [ ]:
len(results_x)

In [ ]:
for i in complete_list:
    print(i)

In [ ]:
import defs_np021
def collect_results(result):
    results.append( result)
results = []

In [ ]:
i = complete_list[1]

In [ ]:
atb = defs_np.process_time_of_day(i[0], i[1], i[2],
                               i[3], i[4], i[5], 
                               i[6], i[7], i[8], i[9],i[10], i[11], i[12],i[13],)

In [ ]:
atb[0].shape, atb[1].shape

In [ ]:
import defs_np019
results_x = []
results_y = []
def collect_results(result):
    results_x.append(result[0])
    results_y.append(result[1])
if __name__ == '__main__':
    start_time = time.time()
    pool = multiprocessing.Pool(processes = multiprocessing.cpu_count() -5)
    print('yes')
    for i in complete_list:
        try:
            result = pool.apply_async(defs_np019.process_time_of_day, args = [i[0], i[1], i[2],
                           i[3], i[4], i[5], 
                           i[6], i[7], i[8], i[9],i[10], i[11],], callback = collect_results)
            print('yes')
        except:
            print(f"Error in worker process: {e}")
        print('O.K.')
    pool.close()
    pool.join()
xTrain1final = np.concatenate(results_x)
yTrain1final = np.concatenate(results_y)

In [ ]:
if __name__ == '__main__':
    start_time = time.time()
    pool = multiprocessing.Pool(processes = multiprocessing.cpu_count() -5)
    print('yes')
    for i in complete_list:
        try:
            result = pool.apply_async(defs_np019.process_time_of_day, args = [i[0], i[1], i[2],
                           i[3], i[4], i[5], 
                           i[6], i[7], i[8], i[9],i[10], i[11],], callback = collect_results)
            print('yes')
        except:
            print(f"Error in worker process: {e}")
        print('O.K.')
    pool.close()
    pool.join()
xTrain1final = np.concatenate(results_x)
yTrain1final = np.concatenate(results_y)

In [ ]:
xTrain1final = np.concatenate(results_x)
yTrain1final = np.concatenate(results_y)

In [ ]:
np.diag(np.sum(W_temp_complete.toarray(), axis=1))

In [ ]:
def process_time_of_day(time_of_day):
    for iteration in range(0, 25):
        A_cp = cp.asarray(A.copy())
        if iteration == 0:
            rolling_window_array_complete = cp.asarray(current_day_OO[:, time_of_day-rolling_window:time_of_day])
            V_int = cp.random.rand(rolling_window_array_complete.shape[1],A.shape[1])
            W_temp_complete = W[:, (time_of_day - rolling_window)* Tens.shape[0]: time_of_day* Tens.shape[0]]
            W_temp_complete = W_temp_complete[(time_of_day - rolling_window)* Tens.shape[0]: time_of_day* Tens.shape[0], :]
            Deg_temp_complete = Deg[:, (time_of_day - rolling_window)* Tens.shape[0]: time_of_day* Tens.shape[0]]
            Deg_temp_complete = Deg_temp_complete[(time_of_day - rolling_window)* Tens.shape[0]: time_of_day* Tens.shape[0], :]
        random_numbers = sorted([random.randint(0, Tens.shape[0]-1) for _ in range(60)])
        list_final = []
        for ii in range (0, rolling_window):
            for jj in random_numbers:
                list_final.append(Tens.shape[0] * ii + jj)
        A_cp = A_cp[random_numbers, :]
        In, I_dash, I_dash2 = generate_graph_functions_outside_loop(A_cp, length = rolling_window, future_time_steps= future_time_steps)
        rolling_window_array =  rolling_window_array_complete[random_numbers, :]
        W_temp = W_temp_complete[list_final, :]
        W_temp = W_temp[:, list_final]
        Deg_temp = csr_matrix(cp.diag(cp.sum(W_temp.toarray(), axis=1).ravel()))
        #In, I_dash, I_dash2 = generate_graph_functions_outside_loop(A_cp, length = rolling_window, future_time_steps= future_time_steps)
        kron_product = generate_graph_functions_inside_loop_once(In, W_temp)
        if iteration >=0:
            Ip, Xi_rep = generate_graph_functions_inside_loop(V_int, In, A_cp)
            neum_term1 = rolling_window_array.T.dot(A_cp) + hy1 * derivative_dynamic_graph_B_W_cp(A_cp, V_int, W_temp, In, Ip, I_dash, kron_product, Xi_rep)
            denom_term1 = (V_int.dot(A_cp.T)).dot(A_cp) + hy1 * derivative_dynamic_graph_B_D_cp(A_cp, V_int, Deg_temp, In, Ip, Xi_rep)
            V_int = V_int * (neum_term1 + e )/ (denom_term1 + e)
    print(time_of_day, time.time() - a, calculate_rmse(rolling_window_array_complete, cp.asarray(A.copy()).dot(V_int.T)))
    V_initialize_previous = V_int.copy()
    V = (cp.asarray(mag_A)).dot(V_int.T).T/data_max
    xTrain1 = V[:, :]
    xTrain1 = xTrain1.reshape(1, *xTrain1.shape)
    if flag == 1:
        xTrain1final = cp.concatenate((xTrain1final, xTrain1), axis=0)
    elif flag == 0:
        xTrain1final = xTrain1.copy()
        

In [ ]:
def print_shape():
    while not stop_event.is_set():
        print("Shape of xTrain1final:", xTrain1final.shape)
        time.sleep(10)
        
def process_time_of_day(time_of_day):
    flag = 0
    for iteration in range(0, 25):
        A_cp = cp.asarray(A.copy())
        if iteration == 0:
            rolling_window_array_complete = cp.asarray(current_day_OO[:, time_of_day-rolling_window:time_of_day])
            V_int = cp.random.rand(rolling_window_array_complete.shape[1],A.shape[1])
            W_temp_complete = W[:, (time_of_day - rolling_window)* Tens.shape[0]: time_of_day* Tens.shape[0]]
            W_temp_complete = W_temp_complete[(time_of_day - rolling_window)* Tens.shape[0]: time_of_day* Tens.shape[0], :]
            Deg_temp_complete = Deg[:, (time_of_day - rolling_window)* Tens.shape[0]: time_of_day* Tens.shape[0]]
            Deg_temp_complete = Deg_temp_complete[(time_of_day - rolling_window)* Tens.shape[0]: time_of_day* Tens.shape[0], :]
        random_numbers = sorted([random.randint(0, Tens.shape[0]-1) for _ in range(60)])
        list_final = []
        for ii in range (0, rolling_window):
            for jj in random_numbers:
                list_final.append(Tens.shape[0] * ii + jj)
        A_cp = A_cp[random_numbers, :]
        In, I_dash, I_dash2 = generate_graph_functions_outside_loop(A_cp, length = rolling_window, future_time_steps= future_time_steps)
        rolling_window_array =  rolling_window_array_complete[random_numbers, :]
        W_temp = W_temp_complete[list_final, :]
        W_temp = W_temp[:, list_final]
        Deg_temp = csr_matrix(cp.diag(cp.sum(W_temp.toarray(), axis=1).ravel()))
        #In, I_dash, I_dash2 = generate_graph_functions_outside_loop(A_cp, length = rolling_window, future_time_steps= future_time_steps)
        kron_product = generate_graph_functions_inside_loop_once(In, W_temp)
        if iteration >=0:
            Ip, Xi_rep = generate_graph_functions_inside_loop(V_int, In, A_cp)
            neum_term1 = rolling_window_array.T.dot(A_cp) + hy1 * derivative_dynamic_graph_B_W_cp(A_cp, V_int, W_temp, In, Ip, I_dash, kron_product, Xi_rep)
            denom_term1 = (V_int.dot(A_cp.T)).dot(A_cp) + hy1 * derivative_dynamic_graph_B_D_cp(A_cp, V_int, Deg_temp, In, Ip, Xi_rep)
            V_int = V_int * (neum_term1 + e )/ (denom_term1 + e)
    print(time_of_day, time.time() - a, calculate_rmse(rolling_window_array_complete, cp.asarray(A.copy()).dot(V_int.T)))
    V_initialize_previous = V_int.copy()
    V = (cp.asarray(mag_A)).dot(V_int.T).T/data_max
    xTrain1 = V[:, :]
    xTrain1 = xTrain1.reshape(1, *xTrain1.shape)
    if flag == 1:
        xTrain1final = np.concatenate((xTrain1final, xTrain1), axis=0)
    elif flag == 0:
        xTrain1final = xTrain1.copy()
    return(xTrain1final)

In [ ]:
for iteration in range(0, 40):
    A_cp = cp.asarray(A.copy())
    if iteration == 0:
        rolling_window_array = cp.asarray(current_day_OO[:, time_of_day-rolling_window:time_of_day + future_time_steps])
        V_int = cp.random.rand(rolling_window_array.shape[1],r)
        V_int[0:rolling_window, ] = V_initialize_previous
        V_int_prev = V_int.copy()
        W_temp_complete = W[:, (time_of_day - rolling_window)* Tens.shape[0]: (time_of_day +future_time_steps) * Tens.shape[0]]
        W_temp_complete = W_temp_complete[(time_of_day - rolling_window)* Tens.shape[0]: (time_of_day +future_time_steps) * Tens.shape[0], :]
        Deg_temp_complete = Deg_complete[:, (time_of_day - rolling_window)* Tens.shape[0]: (time_of_day +future_time_steps) * Tens.shape[0]]
        Deg_temp = Deg_temp_complete[(time_of_day - rolling_window)* Tens.shape[0]: (time_of_day +future_time_steps) * Tens.shape[0], :]
        kron_product = generate_graph_functions_inside_loop_once(In, W_temp)
        Weight_prev = (V_int + 0.0001)/(V_int + 0.0001)
        Weight_prev[time_of_day:] = 0
    if iteration >=0:
        Ip, Xi_rep = generate_graph_functions_inside_loop(V_int, In, A_cp)
        neum_term1 = (rolling_window_array.T.dot(A_cp) 
                      + hy1 * derivative_dynamic_graph_B_W_cp(A_cp, V_int, W_temp, In, Ip, I_dash2, kron_product, Xi_rep)
                     + hy_prev * Weight_prev * V_int_prev)
        denom_term1 = ((V_int.dot(A_cp.T)).dot(A_cp) 
                       + hy1 * derivative_dynamic_graph_B_D_cp(A_cp, V_int, Deg_temp, In, Ip, Xi_rep)
                      + hy_prev * Weight_prev * V_int)
        V_int = V_int * (neum_term1 + e )/ (denom_term1 + e)
V = (cp.asarray(mag_A)).dot(V_int.T).T / data_max
xPred1 = V[:, :]
xPred1 = xPred1.reshape(1, *xPred1.shape)
if flag == 1:
    yTrain1final = np.concatenate((yTrain1final, xPred1), axis=0)
elif flag == 0:
    yTrain1final = xPred1.copy()
    flag = 1

10. Import data

In [ ]:
import os
name= 'Logan'
Data_directory =os.getcwd()

Data_directory = r'\\hpc-fs.qut.edu.au\n10680535\Paper_5\Datasets'
from scipy import sparse
Tens = np.load(Data_directory + os.sep + 'Train_' + name + '_3D_data.npy')
Test_Tens = np.load(Data_directory + os.sep + 'Test_' + name + '_3D_data.npy')
test_dataset = pd.read_csv(Data_directory + os.sep + 'test_' + name + '_data.csv', index_col = [0,1], header = [0])
Val_Tens = np.load(Data_directory + os.sep + 'Val_' + name + '_3D_data.npy')
val_dataset = pd.read_csv(Data_directory + os.sep + 'Val_' + name + '_data.csv', index_col = [0,1], header = [0])
ind = pd.read_csv(Data_directory + os.sep + 'date_time_train.csv', index_col = [0,1])
stl_clm = pd.read_csv(Data_directory + os.sep + 'section_names.csv', index_col = 0)
train_data_shubham= pd.read_csv(Data_directory + os.sep + r'Train_' + name + '_data.csv', index_col = [0,1], header = [0])
W = sparse.load_npz(Data_directory + os.sep + 'Dynamic_Adjacency_' + name+'.npz')
#W = np.load(Data_directory + os.sep + 'Static_Adjacency_'+ name + '_Topo_Traffic.npy')
MWY_Sections = pd.read_csv(Data_directory + os.sep + r'MWY_Sections.csv')
#section_ids = MWY_Sections['Sections'].tolist()  # Get the list of section IDs'''
section_ids = train_data_shubham.columns.to_list()
serial_numbers = [test_dataset.columns.get_loc(section_id)  for section_id in section_ids]
Tens = Tens[serial_numbers, :, :]
Test_Tens = Test_Tens[serial_numbers, :, :]
test_dataset = pd.read_csv(Data_directory + os.sep + 'test_' + name + '_data.csv', index_col = [0,1], header = [0])
test_dataset = test_dataset.T.iloc[serial_numbers].T
Val_Tens = Val_Tens[serial_numbers, :, :]
val_dataset = pd.read_csv(Data_directory + os.sep + 'Val_' + name + '_data.csv', index_col = [0,1], header = [0])
val_dataset = val_dataset.T.iloc[serial_numbers].T
stl_clm = stl_clm.iloc[serial_numbers]
train_data_shubham = train_data_shubham.T.iloc[serial_numbers].T
stl_clm.index = stl_clm.index.astype('str')
test_dataset = test_dataset[stl_clm.index].T
test_data_prior = test_dataset.copy(deep = True)
val_dataset = val_dataset[stl_clm.index].T
val_data_prior = val_dataset.copy(deep = True)
test_dataset_col =  test_dataset.columns
val_dataset_col =  val_dataset.columns
train_data_shubham = train_data_shubham[test_data_prior.index.astype(str)]
train_data_shubham_prior = train_data_shubham.T
serial_numbers_W = serial_numbers.copy()
for i in range(1, 96):
    for j in serial_numbers:
        serial_numbers_W.append(i*476 + j)
#W= coo_matrix(W.toarray()[serial_numbers_W].T[serial_numbers_W].T)
W = coo_matrix(W)
W_dense = W.toarray()
'''# Create a heatmap plot of the dense matrix
plt.imshow(W_dense, cmap='hot', interpolation='nearest')
plt.colorbar()  # Add colorbar for reference
plt.title("Heatmap of W")
plt.xlabel("Columns")
plt.ylabel("Rows")
plt.show()'''
W = (W + W.T)/2
W.setdiag(0, k=0)
Deg = sparse.diags(np.squeeze(np.asarray(W.sum(axis = 0)))).tocsr()
W.setdiag(0, k=0)
Deg = sparse.diags(np.squeeze(np.asarray(W.sum(axis = 0)))).tocsr()
file_path = Data_directory + os.sep + 'Complete_Logan_3D_data.npy'
Complete_Tens = np.load(file_path)
Complete_Tens = Complete_Tens[serial_numbers, :, :]
Tens1 = Tens.transpose(0,2,1)

In [ ]:
#link = r'\\hpc-fs.qut.edu.au\n10680535\Paper_5\5'
A = np.load('A.npy')
B = np.load('B.npy')
C = np.load('C.npy')
A.shape

In [ ]:
rec_df = pd.DataFrame(A.dot(linalg.khatri_rao(B, C).T), 
                      index = train_data_shubham.columns, columns = train_data_shubham.index).T
train_data_shubham_prior.T['11780201_st0'].iloc[0:100].plot()
rec_df['11780201_st0'].iloc[0:100].plot(color = 'red')

In [ ]:
mag_A = np.diag(np.asarray(np.sqrt((A.T.dot(A)).sum(axis=1))).reshape(-1))
mag_A_inverse = np.linalg.inv(mag_A)
data = pd.DataFrame((mag_A).dot(linalg.khatri_rao(B, C).T).T, index=ind.index).round(5)
data_max = np.array(data).max()
rank = r = A.shape[1]
rank

In [ ]:
#Step 2: Pattern mining
Patterns_B, Patterns = generate_patterns_and_factors(train_data_shubham_prior, B, rank=rank, alpha=0, alpha2=0.1, e=1E-10, n_iter=500, eps=2000, MinPts=4, ssims=[0.005])

In [ ]:
flag = 0
rolling_window = 20
future_time_steps = 4
last_time = 88
hyper_smooth = 0
weight_future = 0.001
hyp_x =hyp_previous_reg= 2000
weight_current_pred = 20000

In [ ]:
Offline_encoding_graph_stochastic(Tens, train_data_shubham, A, B, C, Deg, W, ind,rolling_window, future_time_steps, hyper_smooth, 
                      hyp_previous_reg , weight_future, weight_current_pred,last_time = 88,
                      yTrain1final = None, xTrain1final = None, day_max = 20, hy1 = 0.01)

In [ ]:
yTrain1final_g2, xTrain1final_g2 = Offline_encoding(Tens, train_data_shubham, A, B, C, 
                                                    ind,rolling_window, future_time_steps,
                                                    hyper_smooth, hyp_previous_reg,
                                                    weight_future, weight_current_pred,
                                                    last_time = 88, yTrain1final = None, 
                                                    xTrain1final = None, day_max = 270)

In [ ]:
yTrain1final_g2, xTrain1final_g2 = Offline_encoding(Tens, train_data_shubham, 
                                                    A, B, C, 
                                                    ind, rolling_window, 
                                                    future_time_steps, 
                                                    hyper_smooth, hyp_previous_reg ,
                                                    weight_future, 
                                                    weight_current_pred, last_time = 88,
                                                    yTrain1final = yTrain1final_g2, 
                                                    xTrain1final = xTrain1final_g2, 
                                                    day_max = 270, )

np.save('yTrain1final_g.npy', yTrain1final_g)
np.save('xTrain1final_g.npy', xTrain1final_g)
np.save('yVal1final_g.npy', yTrain1final_g)
np.save('xVal1final_g.npy', xTrain1final_g)
np.save('A.npy', A)
np.save('B.npy', B)
np.save('C.npy', C)

In [ ]:
val_data_ind = test_data_prior.copy(deep = True).T
val_data_ind['a'] = 0
val_data_ind = val_data_ind[['a']]
val_data_ind, Val_Tens.shape
yVal1final_g1, xVal1final_g1 = Offline_encoding (Test_Tens,test_data_prior.T, A, B, C, 
                                                 val_data_ind,rolling_window, future_time_steps, 
                                                 hyper_smooth, hyp_previous_reg , weight_future, 
                                                 weight_current_pred,last_time = 88,
                                                 yTrain1final = None, xTrain1final = None, day_max = 5)

In [ ]:
yVal1final_g1, xVal1final_g1 = Offline_encoding (Test_Tens,test_data_prior.T, A, B, C, 
                                                 val_data_ind,rolling_window, future_time_steps, 
                                                 hyper_smooth, hyp_previous_reg , weight_future,
                                                 weight_current_pred,last_time = 88, yTrain1final = yVal1final_g1, 
                                                 xTrain1final = xVal1final_g1, day_max = 30)

In [ ]:
np.save('yTrain1final_g2.npy', yTrain1final_g2)
np.save('xTrain1final_g2.npy', xTrain1final_g2)
np.save('yVal1final_g1.npy', yVal1final_g1)
np.save('xVal1final_g1.npy', xVal1final_g1)

In [ ]:
# Provide the values for rolling_window and future_time_steps
rolling_window = 12
future_time_steps = 4
Index_final, yTrain1true_Original1 = Generate_original_sequence_arrays(Tens, rolling_window = 12, 
                                                                       future_time_steps = 4,
                                                                       max_days = 270, 
                                                                       last_time = 88)
yTrain1true_Original1.shape

In [ ]:
len(Index_final)

In [ ]:
# Provide the values for rolling_window and future_time_steps
rolling_window = 12
future_time_steps = 4
Index_final_val, yVal1true_Original1 = Generate_original_sequence_arrays(Test_Tens, 
                                                                         rolling_window = 12,
                                                                         future_time_steps = 4, 
                                                                         max_days = 30, last_time = 88)

In [ ]:
yVal1true_Original1.shape,yTrain1true_Original1.shape

In [ ]:
np.save('Index_final.npy', Index_final)
np.save('yTrain1true_Original1.npy', yTrain1true_Original1)
np.save('Index_final_val.npy', Index_final_val)
np.save('yVal1true_Original1.npy', yVal1true_Original1)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Example loop to generate plots
for i in range(0, 10):
    fig, ax = plt.subplots()

    y_train_df = pd.DataFrame(xTrain1final[20])
    x_train_df = pd.DataFrame(yTrain1final[20])

    pd.DataFrame({'V-output sequence': y_train_df[i]}).plot(ax=ax, color='red', label='V-output sequence')
    pd.DataFrame({'V-output sequence': x_train_df[i]}).plot(ax=ax, label='V-input sequence')
    
    plt.xlabel('time-period')  # Set x-axis label
    plt.ylabel('coefficient')  # Set y-axis label
    plt.title('Sequences')  # Set plot title
    plt.legend()  # Display legend
    
    plt.show()


In [ ]:
xTrain1final_g2.shape, len(Index_final)